## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback

In [ ]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.sample(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
33620,T3,CompanySponsored,C1000,Preservation,Trust,1,100000-499999,N,60951,0
16101,T3,CompanySponsored,C2100,Preservation,Trust,1,100000-499999,N,5332,0
28305,T3,Independent,C2000,Preservation,Trust,1,0,N,5000,1
28481,T3,CompanySponsored,C1200,Preservation,Trust,1,0,N,5000,1
33004,T6,Independent,C1000,ProductDev,Trust,1,0,N,5000,1
23740,T3,Independent,C2000,Preservation,Trust,1,25000-99999,N,11287,0
7871,T3,Independent,C1000,Preservation,Trust,1,0,N,5000,0
9342,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
7889,T3,Independent,C1000,Preservation,Trust,1,1M-5M,N,1116861,1
28583,T3,CompanySponsored,C1200,Preservation,Trust,1,0,N,5000,0


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_1 = class_counts.loc[class_counts > 1]
class_counts_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df['ASK_AMT'].value_counts()
ask_amt_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [ ]:
# You may find it helpful to look at ASK_AMT value counts >1
ask_amt = ask_amt_counts.loc[ask_amt_counts > 1]
ask_amt

5000      25398
10478         3
15583         3
63981         3
6725          3
          ...  
100000        2
8181          2
24746         2
45793         2
240780        2
Name: ASK_AMT, Length: 152, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of ask_amt to be replaced
# use the variable name `ask_amt_to_replace`
ask_amt_to_replace = list(ask_amt_counts[ask_amt_counts < 3].index)

# Replace in dataframe
for amt in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"Other")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8889
15583        3
6725         3
10478        3
63981        3
Name: ASK_AMT, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
app_final = pd.get_dummies(application_df)
app_final.sample(10)

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_6725,ASK_AMT_10478,ASK_AMT_15583,ASK_AMT_63981,ASK_AMT_Other
28465,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
33343,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3863,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
5013,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
20609,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
29593,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
28454,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
20719,1,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
22370,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
32299,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = app_final["IS_SUCCESSFUL"]
X = app_final.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size = 0.7)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.values[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 30

nn_model = Sequential()

# First hidden layer
nn_model.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(Dense(units=hidden_nodes_layer2, activation="relu"))
nn_model.add(Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn_model.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                490       
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 30)                630       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,371
Trainable params: 1,371
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#  Create a callback that saves the model's weights every five epochs

class SaveWeightsCallback(Callback):
    def __init__(self):
        super(SaveWeightsCallback, self).__init__()
        self.epoch_count = 0

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_count += 1
        if self.epoch_count % 5 == 0:
            self.model.save_weights(f'AlphabetSoup_weights_epoch_{self.epoch_count}.h5')
            print(f'Saved model weights at epoch {self.epoch_count}.')


In [ ]:
# Train the model
save_weights_callback = SaveWeightsCallback()
nn_model.fit(X_train_scaled, y_train, epochs=50, callbacks=[save_weights_callback])

Epoch 1/50
751/751 [==============================] - 4s 3ms/step - loss: 0.5982 - accuracy: 0.6957
Epoch 2/50
751/751 [==============================] - 2s 2ms/step - loss: 0.5626 - accuracy: 0.7249
Epoch 3/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5577 - accuracy: 0.7266
Epoch 4/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7279
Epoch 5/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7296
Epoch 6/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7299
Epoch 7/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7312
Epoch 8/50
751/751 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7303
Epoch 9/50
751/751 [==============================] - 2s 3ms/step - loss: 0.5504 - accuracy: 0.7316
Epoch 10/50
751/751 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7319

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - loss: 0.5475 - accuracy: 0.7305 - 669ms/epoch - 2ms/step
Loss: 0.5475379228591919, Accuracy: 0.7305150628089905


In [ ]:
# Export our model to HDF5 file
nn_model.save("AphabetSoup_Charity.h5")
